In [33]:
import os
from numba import jit
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import scipy
import pandas as pd
import xgboost as xgb

In [34]:
rcParams['font.family']='Times New Roman'

In [35]:
tb = pd.read_csv('./results/all_features.csv')
tb

,1D Gray Entropy,2D Gray Entropy,1D Hue Entropy,Color Richness,Area,Horizontal Intervals,Vertical Intervals,Dominant X,Dominant Y,Dominant Z,label
0,3.757319,8.199638,2.984306,4.0,0.180172,0.000000,0.006129,-0.022779,-0.031353,0.823529,1
1,4.714888,9.984443,4.135668,3.0,0.983521,0.001712,0.003144,-0.045593,-0.102404,0.772549,1
2,5.070659,10.304571,4.157235,4.0,0.995805,0.002059,0.004746,-0.014302,0.204529,0.396078,1
3,4.861554,9.968199,4.632747,4.0,0.995803,0.001441,0.002214,-0.034903,-0.078394,0.235294,1
4,4.831612,9.891949,4.300436,2.0,0.976716,0.001335,0.003850,-0.122485,0.230361,0.458824,1
...,...,...,...,...,...,...,...,...,...,...,...
530,4.472156,9.025126,3.321349,7.0,0.891165,0.000029,0.000040,-0.015255,-0.046950,0.839216,0
531,4.658013,8.932943,3.891208,3.0,0.931654,0.000442,0.000485,-0.009792,-0.034149,0.823529,0
532,4.726424,8.967950,4.560189,7.0,0.973873,0.003282,0.004204,0.058039,0.000000,0.870588,0
533,4.792121,9.414940,4.616940,5.0,0.973879,0.000788,0.001320,-0.032092,0.028895,0.211765,0


In [38]:
abla_groups = [[0,1],[2,3,7,8,9],[4,5,6]]

for s_num in range(0,3):
    for abla_comb in itertools.combinations([0,1,2],s_num):
        print(abla_comb)
        abla_group = []
        for idx in abla_comb:
            abla_group += abla_groups[idx]
        sub_tb = tb.drop(tb.columns[abla_group], axis=1)

        accuracy = []
        precision = []
        recall = []
        F1 = []
        AUC = []

        kf = KFold(n_splits=5, shuffle=True, random_state=19260817)
        for train_index, test_index in kf.split(sub_tb):
            train_X = sub_tb.iloc[:,:-1].iloc[train_index]
            test_X = sub_tb.iloc[:,:-1].iloc[test_index]
            train_y = sub_tb.iloc[:,-1][train_index]
            test_y = sub_tb.iloc[:,-1][test_index]

            svm_clf = svm.SVC(kernel='rbf', gamma=0.6, probability=True)
            svm_clf.fit(train_X,train_y)
            test_pred = svm_clf.predict(test_X)
            test_prob = svm_clf.predict_proba(test_X)
            fpr, tpr, thresholds = roc_curve(test_y,test_prob[:,1], pos_label=1)

            accuracy.append(accuracy_score(test_y, test_pred))
            precision.append(precision_score(test_y, test_pred))
            recall.append(recall_score(test_y, test_pred))
            F1.append(f1_score(test_y, test_pred))
            AUC.append(auc(fpr,tpr))
        print(np.mean(accuracy), np.mean(precision), np.mean(recall), np.mean(F1), np.mean(AUC))
        print(np.std(accuracy), np.std(precision), np.std(recall), np.std(F1), np.std(AUC))
        print()

()
0.8598130841121495 0.864000465990934 0.9492866771347783 0.9045521932880399 0.9090444557926007
0.01182159872960142 0.014011416593053978 0.0103586902862562 0.009095546653007925 0.0283678562148418

(0,)
0.6785046728971962 0.7117168868010751 0.9178371653055197 0.7988814340300471 0.6646952737197173
0.02682747681197631 0.02209535935080497 0.08436591136610877 0.02267264788906891 0.07038313651003054

(1,)
0.8504672897196262 0.8570168110737535 0.944298461070613 0.8984176388015992 0.9127491384999024
0.01182159872960142 0.010046268678324498 0.016738127204037157 0.008067318175168432 0.027810770640787227

(2,)
0.8523364485981307 0.8577031610364945 0.9464764423625182 0.899798821281038 0.908256651524875
0.013735456501587915 0.014086760146807768 0.012488756932156191 0.009270586146130886 0.027394395042853927

(0, 1)
0.7009345794392523 0.7009345794392523 1.0 0.8238647428663448 0.7521006304834328
0.027724106493815567 0.027724106493815567 0.0 0.019086065716924103 0.036805049904183186

(0, 2)
0.67850467

In [40]:
abla_groups = [[0,1],[2,3,7,8,9],[4,5,6]]

for s_num in range(0,3):
    for abla_comb in itertools.combinations([0,1,2],s_num):
        print(abla_comb)
        abla_group = []
        for idx in abla_comb:
            abla_group += abla_groups[idx]
        sub_tb = tb.drop(tb.columns[abla_group], axis=1)

        accuracy = []
        precision = []
        recall = []
        F1 = []
        AUC = []

        kf = KFold(n_splits=5, shuffle=True, random_state=19260817)
        for train_index, test_index in kf.split(sub_tb):
            train_X = sub_tb.iloc[:,:-1].iloc[train_index]
            test_X = sub_tb.iloc[:,:-1].iloc[test_index]
            train_y = sub_tb.iloc[:,-1][train_index]
            test_y = sub_tb.iloc[:,-1][test_index]

            dtrain = xgb.DMatrix(train_X,train_y)
            dtest = xgb.DMatrix(test_X,test_y)

            param = {'max_depth':5, 'eta':0.1, 'verbosity':1, 'objective':'binary:logistic'}
            tree = xgb.train(param, dtrain, num_boost_round=20)
            test_pred = tree.predict(dtest)
            fpr, tpr, thresholds = roc_curve(test_y,test_pred, pos_label=1)
            for i in range(len(test_pred)):
                if test_pred[i] > 0.5:
                     test_pred[i]=1
                else:
                    test_pred[i]=0

            accuracy.append(accuracy_score(test_y, test_pred))
            precision.append(precision_score(test_y, test_pred))
            recall.append(recall_score(test_y, test_pred))
            F1.append(f1_score(test_y, test_pred))
            AUC.append(auc(fpr,tpr))
        print(np.mean(accuracy), np.mean(precision), np.mean(recall), np.mean(F1), np.mean(AUC))
        print(np.std(accuracy), np.std(precision), np.std(recall), np.std(F1), np.std(AUC))
        print()

()
0.9289719626168225 0.9377998919850536 0.9627468266708773 0.9499213013392296 0.9800141831623718
0.013987504249622211 0.01445508850922546 0.019244255126820424 0.010355946044673325 0.006129823011965103

(0,)
0.9196261682242991 0.9232157861964175 0.9652474041714548 0.9436530764753451 0.9704024457079891
0.012677252304907044 0.015147340846119621 0.014172894557631042 0.010470175577342594 0.007299083292750663

(1,)
0.9233644859813085 0.9371645344146933 0.9544885684126191 0.9455783295891587 0.9692848118917521
0.013735456501587915 0.014642747663910539 0.018345090858892642 0.010793000666389918 0.01548921269474433

(2,)
0.8728971962616822 0.8777083725318777 0.9526730235590997 0.9131039372548748 0.902887753065623
0.011214953271028038 0.016390159055191318 0.029444204872538977 0.006268300391787711 0.03077888876716878

(0, 1)
0.8897196261682243 0.9124236976058958 0.9331961221201727 0.9217621194282243 0.9526932774684849
0.019061755191001065 0.03271801387870248 0.03412581888187252 0.01567446732765173